In [ ]:
# File: woven.ipynb
# Code: Claude Code and Codex
# Review: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
import random

import numpy as np
from frontend import App


# function to generate woven cylinder structure with interlaced strands
def make_woven_cylinder(n: int, offset: float, scale: float):
    dx, width = 1.0 / (n - 1), 1.25
    scale = 2.0 * 1.48 * scale
    v_steps = int(25.0 * scale)
    sep, strands = 0.5, []

    # create vertical strands
    for i in range(v_steps):
        theta = 2.0 * np.pi * i / v_steps
        xyz = np.zeros((n, 3))
        xyz[:, 0] = width * (2.0 * dx * np.arange(n) - 1.0)
        xyz[:, 1], xyz[:, 2] = sep * np.sin(theta), sep * np.cos(theta)
        strands.append((xyz, False))

    # create horizontal ring strands with wave pattern
    h_steps = int(30.0 * scale)
    ring_steps = v_steps * 3
    assert ring_steps % 2 == 0, "ring_steps must be even"
    amp, dx_h, half_v = 1.2 * offset, 1.0 / (h_steps - 1), v_steps // 2

    for i in range(1, h_steps - 1):
        sgn = 1.0 if (i % 2 == 0) else -1.0
        xyz = np.zeros((ring_steps, 3))
        xyz[:, 0] = width * (2.0 * dx_h * i - 1.0)
        j_indices = np.arange(ring_steps)
        theta_vals = 2.0 * np.pi * j_indices / ring_steps
        r = sep + sgn * amp * np.cos(half_v * theta_vals)
        xyz[:, 1], xyz[:, 2] = r * np.sin(theta_vals), r * np.cos(theta_vals)
        strands.append((xyz, True))

    return strands


# create an app
app = App.create("woven")

# create a scene
scene = app.scene.create()

# generate and add all woven strands
angular_vel, move_delta, t_end = 360 + 10 * random.random(), 0.15, 10
for k, (V, closed) in enumerate(make_woven_cylinder(256, 4e-3, 2.0)):
    # create edge connectivity for rod strands
    E = [[i, i + 1] for i in range(len(V) - 1)]
    if closed:
        E.append([len(V) - 1, 0])
    name = f"strand-{k}"
    app.asset.add.rod(name, V, np.array(E, dtype=np.uint32))
    obj = scene.add(name)
    # set material properties for woven strands
    (
        obj.param.set("bend", 1e-3)
        .set("young-mod", 1e5)
        .set("contact-gap", 1.5e-3)
        .set("contact-offset", 2e-3)
        .set("friction", 0.01)
        .set("length-factor", 0.8)
    )
    # pin and animate open-ended strands
    if not closed:
        (
            obj.pin(obj.grab([-1, 0, 0]))
            .spin(axis=[1, 0, 0], angular_velocity=angular_vel)
            .move_by([move_delta, 0, 0], 0, t_end)
        )
        (
            obj.pin(obj.grab([1, 0, 0]))
            .spin(axis=[-1, 0, 0], angular_velocity=angular_vel)
            .move_by([-move_delta, 0, 0], 0, t_end)
        )

# compile the scene and report stats
scene = scene.build().report()

# preview the initial scene
scene.preview()

In [ ]:
# create a new session with the compiled scene
session = app.session.create(scene)

# set session parameters - disable gravity and add air friction
(
    session.param.set("frames", 450)
    .set("dt", 1 / 120)
    .set("gravity", 0.0)
    .set("isotropic-air-friction", 1e-3)
)

# build this session
session = session.build()

In [ ]:
# start the simulation and live-preview the results
session.start().preview()

# also show simulation logs in realtime
session.stream()

In [ ]:
# create an animation from the simulation results
session.animate()

In [ ]:
# export the animation to file
session.export.animation()

In [ ]:
# this is for CI
if app.ci:
    assert session.finished()